# 串（String）总结

## 基本概念
- **定义**：串（String）是由零个或多个字符组成的有限序列。  
  - 空串：长度为 0 的串，记为 `""`。  
  - 子串：串中任意连续的一段。  
  - 主串：包含子串的串。  
  - 串相等：当且仅当两个串长度相等，且对应位置的字符完全相同。  
  - **前缀（Prefix）**：主串从起始位置开始的任意连续子串。  
    - 例如：主串 `S = "abcd"`，其前缀有 `"a"`, `"ab"`, `"abc"`, `"abcd"`。  
  - **后缀（Suffix）**：主串从任意位置到末尾的连续子串。  
    - 例如：主串 `S = "abcd"`，其后缀有 `"d"`, `"cd"`, `"bcd"`, `"abcd"`。  
  - **真前缀/真后缀**：前缀或后缀不包括整个串本身。  
## 串的存储结构
1. **定长顺序存储**
   - 用固定长度的数组存储，可能存在空间浪费。  

2. **堆分配存储**
   - 动态分配空间，长度可变。  

3. **链式存储**
   - 用链表存储串中的字符，便于插入删除，但存储密度低。  

## 串的模式匹配算法
1. **朴素模式匹配算法（Brute Force, BF）**
   - 从主串的每个位置开始，逐个字符比较，直到找到或失败。  
   - 时间复杂度：O(m·n)，其中 m、n 分别为主串和模式串长度。  

2. **KMP 算法(重点)**
   - 基于部分匹配表（next 数组），避免无效回溯。  
   - 时间复杂度：O(m+n)。 

## KMP算法详解
- Next表：通过查表快速确定模式串的下一匹配位置
    - ![next](./sources/next.png)
        - j=1时，b的真前缀和真后缀长度为0，共有元素长度为0
        - j=2时，ba的真前缀为b,真后缀为a，共有元素长度为0
        - j=3时，bac的真前缀为{b,ba},真后缀为{c,ac}，共有元素长度为0
        - j=4时，bacb的真前缀为{b,ba,bac},真后缀为{b,cb,acb}，共有元素长度为b，长度为1
        - j=5时，bacba的真前缀为{b,ba,bac,bacb},真后缀为{a,ba,cba,acba}，共有元素长度为{ba}，长度为2
        - j=0时，-1
- ![KMP](./sources/KMP.png)
- 本质：找到相同的真前缀，真后缀来进行剪枝
### 进一步改进
- 引入失配的经验教训：选定的t不能使得P[t]=P[j] ————一步步回退或者直接相等修改next；
    - ![next](./sources/KMP3.png)
    - ![next!](./sources/KMP2.png)

In [12]:
class MyString:
    def __init__(self, content=""):
        # 使用 list 存储字符，方便插入删除
        self.chars = list(content)

    # --- 基本操作 ---
    def __str__(self):
        return "".join(self.chars)

    def __len__(self):
        return len(self.chars)

    def is_empty(self):
        return len(self.chars) == 0

    def clear(self):
        self.chars = []

    def copy(self):
        return MyString("".join(self.chars))

    def compare(self, other):
        """按字典序比较两个串"""
        return (str(self) > str(other)) - (str(self) < str(other))

    # --- 串操作 ---
    def concat(self, other):
        """拼接"""
        return MyString(str(self) + str(other))

    def substring(self, pos, length):
        """取子串：从 pos 开始，长度 length"""
        if pos < 0 or pos + length > len(self.chars):
            raise IndexError("substring out of range")
        return MyString("".join(self.chars[pos:pos + length]))

    def insert(self, pos, sub):
        """在位置 pos 插入 sub"""
        if pos < 0 or pos > len(self.chars):
            raise IndexError("insert position out of range")
        self.chars = self.chars[:pos] + list(str(sub)) + self.chars[pos:]

    def delete(self, pos, length):
        """删除从 pos 开始长度为 length 的子串"""
        if pos < 0 or pos + length > len(self.chars):
            raise IndexError("delete out of range")
        self.chars = self.chars[:pos] + self.chars[pos+length:]

    # 模式匹配
    def index_bf(self, pattern, start=0):
        """朴素模式匹配（BF）"""
        m, n = len(self.chars), len(pattern.chars)
        for i in range(start, m - n + 1):
            if self.chars[i:i+n] == pattern.chars:
                return i
        return -1

    # KMP算法，构造next数组
    def build_next(self, pattern):
        """KMP 构造 next 数组"""
        n = len(pattern.chars)
        next_arr = [0] * n
        j = 0
        for i in range(1, n):
            while j > 0 and pattern.chars[i] != pattern.chars[j]:
                j = next_arr[j - 1]
            if pattern.chars[i] == pattern.chars[j]:
                j += 1
            next_arr[i] = j
        return next_arr

    def build_next_val(self,pattern):
        """KMP 改进版 next 数组（next_val）"""
        n = len(pattern.chars)
        next_val = [0] * n
        j = 0
        next_val[0] = 0

        for i in range(1, n):
            while j > 0 and pattern.chars[i] != pattern.chars[j]:
                j = next_val[j - 1]
            if pattern.chars[i] == pattern.chars[j]:
                j += 1
            # 改进：如果相等，则直接继承 next_val[j-1] 避免重复匹配
            if i + 1 < n and pattern.chars[i + 1] == pattern.chars[j]:
                next_val[i] = next_val[j - 1]
            else:
                next_val[i] = j
        return next_val

    def index_kmp(self, pattern, start=0, val = False):
        """KMP 模式匹配"""
        m, n = len(self.chars), len(pattern.chars)
        if val == False:
            next_arr = self.build_next(pattern)
            print("next_arr:",next_arr)
        else:
            next_arr = self.build_next_val(pattern)
            print("next_arr_val:",next_arr)
        j = 0
        time = 0
        for i in range(start, m):
            while j > 0 and self.chars[i] != pattern.chars[j]:
                j = next_arr[j - 1]
                time += 1
            time += 1
            if self.chars[i] == pattern.chars[j]:
                j += 1
            if j == n:
                print("匹配次数：",time)
                return i - n + 1
        return -1

s1 = MyString("aaacddddssdsiwedjaaaaacddddssd")
s2 = MyString("xyz")

print("s1:", s1)                 
print("拼接:", s1.concat(s2))      
print("子串:", s1.substring(1, 3)) 

s1.insert(2, MyString("123"))
print("插入后:", s1)              

s1.delete(2, 3)
print("删除后:", s1)            

pattern = MyString("bacbab")
print("BF匹配位置:", s1.index_bf(pattern)) 
print("KMP匹配位置:", s1.index_kmp(pattern)) 
print("KMP匹配位置(val):", s1.index_kmp(pattern,val=True)) 

s1: aaacddddssdsiwedjaaaaacddddssd
拼接: aaacddddssdsiwedjaaaaacddddssdxyz
子串: aac
插入后: aa123acddddssdsiwedjaaaaacddddssd
删除后: aaacddddssdsiwedjaaaaacddddssd
BF匹配位置: -1
next_arr: [0, 0, 0, 1, 2, 1]
KMP匹配位置: -1
next_arr_val: [0, 0, 0, 0, 2, 1]
KMP匹配位置(val): -1
